### Chapter 05 트리 알고리즘 ▶️화이트 와인을 찾아라!
#### 05-2 교차 검증과 그리드 서치 ▶️검증 세트가 필요한 이유를 이해하고 교차 검증해 보기

-------------------
#### - train/test set을 기준으로 모델을 개선시키면 결국 test set에 가장 잘 맞는 모델이 만들어 진다. 
#### - k-fold : train/validation/test set을 구성하고 교차&반복 분석을 통해 최상의 모델을 찾는다. 

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 

In [2]:
wine = pd.read_csv("http://bit.ly/wine_csv_data")
wine.describe()

,alcohol,sugar,pH,class
count,6497.000000,6497.000000,6497.000000,6497.000000
mean,10.491801,5.443235,3.218501,0.753886
std,1.192712,4.757804,0.160787,0.430779
min,8.000000,0.600000,2.720000,0.000000
25%,9.500000,1.800000,3.110000,1.000000
50%,10.300000,3.000000,3.210000,1.000000
75%,11.300000,8.100000,3.320000,1.000000
max,14.900000,65.800000,4.010000,1.000000


In [3]:
## 데이터를 6:2:2 비율로 구분하여 훈련시키기 
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)

print( sub_input.shape, val_input.shape, sub_target.shape, val_target.shape )

(4157, 3) (1040, 3) (4157,) (1040,)


In [4]:
from sklearn.tree import DecisionTreeClassifier
dt_1 = DecisionTreeClassifier(random_state = 42)
dt_1.fit(sub_input, sub_target)

print( dt_1.score(sub_input, sub_target) )
print( dt_1.score(val_input, val_target) )

0.9971133028626413
0.864423076923077


- 위 결과는 sub dataset에 overfitting 되었다. k-fold를 써보자. 
-------------------------
- cross_validate() : 사이킷런의 교차검증 함수. default k = 5 이다. 
- StratifiedKFold()를 사용하기도 한다. 회귀모델인 경우 cross_validate()는 StratifiedKFold()를 사용하기 때문에 동일한 결과를 낸다. 

In [5]:
from sklearn.model_selection import cross_validate
score = cross_validate(dt_1, train_input, train_target)
score

{'fit_time': array([0.00499916, 0.00500083, 0.00500107, 0.00500059, 0.00500131]),
 'score_time': array([0.00099969, 0.        , 0.00100088, 0.        , 0.        ]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [6]:
## test_score 값들을 평균하여 모형의 성능(accuracy)를 판단할 수 있다. 
np.mean(score['test_score'])

0.855300214703487

In [7]:
## cross_validate()는 훈련세트를 섞지 않는다. 
## 따라서 실전에서는 StratifiedKFold()를 쓰며, splitter를 사용하여 데이터를 shuffle 시켜준다. 
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
score = cross_validate(dt_1, train_input, train_target, cv = splitter)

print(score)
print('-------------------------------------')
print(np.mean(score['test_score']))

{'fit_time': array([0.00600171, 0.006001  , 0.00500059, 0.00500059, 0.00500059,
       0.00600123, 0.00500202, 0.00500083, 0.00500059, 0.00500035]), 'score_time': array([0.00099945, 0.0010004 , 0.00100064, 0.00100088, 0.        ,
       0.        , 0.00099969, 0.00100064, 0.        , 0.        ]), 'test_score': array([0.83461538, 0.87884615, 0.85384615, 0.85384615, 0.84615385,
       0.87307692, 0.85961538, 0.85549133, 0.85163776, 0.86705202])}
-------------------------------------
0.8574181117533719


----------------------------
- Hyperparameter tunning : GridSearchCV()는 파라미터의 탐색과 교차검증을 한번에 수행한다. 
- GridSearchCV()는 default로 5-fold 교차검증을 수행함. 만약 파라미터가 5개라면 5x5 = 25개 모델을 훈련하게 된다. 
- n_jobs = -1 : 사용될 cpu 코어수를 지정. -1이면 가용한 모든 코어를 사용한다. 

In [8]:
## Decision Tree 모델의 최적의 min_impurity_decrease(지니계수 즉 불순도 감소폭이 얼마 이하이면 훈련을 멈춰라)를 찾는 과정 
from sklearn.model_selection import GridSearchCV
prams = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), prams, n_jobs = -1)
gs.fit(train_input, train_target)

## 훈련된 25개 모델중 best 모델의 결과를 출력하기 
print(gs.best_estimator_)
print(gs.best_params_)

DecisionTreeClassifier(min_impurity_decrease=0.0001, random_state=42)
{'min_impurity_decrease': 0.0001}


In [9]:
dt_best = gs.best_estimator_
dt_best.score(train_input, train_target)

0.9615162593804117

In [10]:
## 5개의 파라미터 각각에 대해 5번씩 교차검증한 결과점수의 평균 
gs.cv_results_['mean_test_score']

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

-------------------------
- 하이퍼파라미터(여기서는 min_impurity_decrease)를 직접 지정하지 않고, scypi를 사용하여 확률분포에서 뽑게 하자. 

In [11]:
## uniform은 지정된 두 숫자사이의 정규분포에서 실수값을, randint는 정수값을 랜덤하게 뽑는다. 
from scipy.stats import uniform, randint 

ugen = uniform(0, 1)
rgen = randint(0, 10)

print( ugen.rvs(10) )
print( rgen.rvs(10) )

[0.38658695 0.50524161 0.45295286 0.53743945 0.92168684 0.18137424
 0.08732816 0.19554629 0.07668446 0.39719227]
[5 1 9 3 8 8 6 2 2 8]


In [12]:
## min_impurity_decrease : 0.0001 ~ 0.001 사이의 실수값을 랜덤 샘플링 
## max_depth : 20~50 사이의 정수 
## min_samples_split : 2~25 사이의 정수 
## min_samples_leaf : 1~25 사이의 정수, 샘플수가 이 값 이하인 경우 더이상 트리를 분할하지 않는다. 

params = {'min_impurity_decrease': uniform(0.0001, 0.001),
         'max_depth': randint(20, 50), 
         'min_samples_split': randint(2, 25), 
         'min_samples_leaf': randint(1, 25)    
         }

In [13]:
from sklearn.model_selection import RandomizedSearchCV
gs_2 = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter = 100, n_jobs = -1, random_state = 42)
gs_2.fit(train_input, train_target)

gs_2.best_params_

{'max_depth': 39,
 'min_impurity_decrease': 0.00034102546602601173,
 'min_samples_leaf': 7,
 'min_samples_split': 13}

In [14]:
## train 데이터로부터 훈련된 모델중 최고 성능의 점수는?
np.max(gs_2.cv_results_['mean_test_score'])

0.8695428296438884

In [15]:
## best 모델로써 test 데이터에 적용시켜보면? 
dt_vf = gs.best_estimator_
dt_vf.score(test_input, test_target)

0.8653846153846154